__собираем статистики по объявлениям__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# import re
#import numpy as np
# from tqdm.notebook import tqdm
import pandas as pd
import geopandas as gpd
# import contextily as ctx
#from shapely.geometry import Polygon

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
# tqdm.pandas()

In [3]:
# !ls -1 data

## считываем данные

In [4]:
data_file = 'data/data_house.pkl'

In [5]:
cols = ['title','adr','latitude','longitude','priceM','ts']
df = pd.read_pickle(data_file)
df['dt'] = pd.to_datetime( df['ts'].dt.date )
print(len(df))
df.sample(3)[cols]

2821


,title,adr,latitude,longitude,priceM,ts
1209,дом 100 м² на участке 2 сот.,"Севастополь, садоводческое товарищество Электрон-2",44.51,33.50,9.50,2022-09-15 11:53:29.976
2423,"дача 50 м² на участке 4,4 сот.","Севастополь, садоводческое товарищество Авторемонтник, 2",44.57,33.49,7.50,2022-09-15 11:53:29.976
85,дом 100 м² на участке 4 сот.,"Севастополь, садоводческое товарищество Надежда",44.58,33.57,6.40,2022-09-15 11:53:29.976


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2821 entries, 0 to 2820
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   avito_id             2821 non-null   int64         
 1   title                2821 non-null   object        
 2   price                2821 non-null   int64         
 3   obj_name             2821 non-null   object        
 4   adr                  2821 non-null   object        
 5   description          2821 non-null   object        
 6   avito_page           2821 non-null   int64         
 7   ts                   2821 non-null   datetime64[ns]
 8   place                2821 non-null   object        
 9   house_area           2821 non-null   float64       
 10  is_part              2821 non-null   bool          
 11  is_townhouse         2821 non-null   bool          
 12  is_SNT               2821 non-null   bool          
 13  land_area            2821 non-nul

In [7]:
df.sample(3)[cols]

,title,adr,latitude,longitude,priceM,ts
2101,"дом 70 м² на участке 1,7 сот.","Севастополь, Матросский пер., 30",44.61,33.51,18.00,2022-09-15 11:53:29.976
52,дом 160 м² на участке 3 сот.,"Севастополь, ул. Челнокова, 9/4",44.59,33.43,23.00,2022-09-15 11:53:29.976
766,"дом 30 м² на участке 6,5 сот.","Севастополь, садоводческое товарищество Горный, 2",44.63,33.58,4.50,2022-09-15 11:53:29.976


In [8]:
# берём объявления с геометкой
print(len(df))
df = df[ (~df['latitude'].isnull()) ].reset_index(drop=True)
print(len(df))

2821
2821


### ограничиваем тип объекта 

In [9]:
print(len(df))
# выкидываем "ущербные" варианты 
# df = df.query('~(is_part|is_SNT|is_townhouse) & (house_area>40.) & (priceM<15.)')
df = df.query('~(is_part) & (house_area>40.) & (priceM<15.)')
print(len(df))

2821
1952


In [10]:
gdf = gpd.GeoDataFrame( df, geometry = gpd.points_from_xy( df['longitude'], df['latitude']), crs='epsg:4326', )
gdf.sample(3)[['title','adr','latitude','longitude','priceM']]

,title,adr,latitude,longitude,priceM
1047,"дом 120 м² на участке 4,2 сот.","Севастополь, ТСН Союз",44.51,33.52,7.50
822,дом 210 м² на участке 9 сот.,"Севастополь, садовое товарищество Маяк-1, 256",44.56,33.45,10.99
1238,дом 90 м² на участке 4 сот.,"Севастополь, садоводческое товарищество Торпеда",44.56,33.47,8.50


In [11]:
del df

In [12]:
cols = ['title','priceM','house_size_category','place','geometry']

# gdf[cols].explore('house_size_category',cmap='rainbow',legend=True,marker_type='marker') 
gdf[cols].explore('house_size_category',cmap='rainbow',legend=True,marker_type='circle_marker') 

In [13]:
cols = ['title','adr','priceM','house_size_category','place','geometry']
gdf[cols].explore('place',cmap='rainbow',legend=True) 

## общая статистика изменение цены

In [14]:
ts = '2022-08-01' # ограничиваем дату

In [15]:
stat = (
    gdf.query('priceM>1.')
    .groupby(['place','house_size_category','dt'])
    ['priceM'].describe(percentiles=[.1,.25,.5,.75,.9])
)
stat

count  mean  std  min  10%  25%  \
place       house_size_category dt                                             
bahchisaray 30-50               2022-09-15   34.00  3.48 1.41 1.60 2.00 2.50   
            50-70               2022-09-15   64.00  4.54 1.89 1.10 2.50 3.18   
            70-150              2022-09-15   76.00  5.95 2.87 1.40 2.52 4.00   
            150-300             2022-09-15   51.00  8.48 3.02 1.79 4.60 6.20   
            300+                2022-09-15    6.00  9.02 3.80 4.80 4.88 5.71   
sevastopol  30-50               2022-09-15  103.00  5.14 1.81 2.00 3.11 3.80   
            50-70               2022-09-15  205.00  6.14 2.40 1.68 3.70 4.50   
            70-150              2022-09-15 1024.00  8.09 2.58 1.50 5.10 6.26   
            150-300             2022-09-15  293.00 10.61 2.80 1.80 6.99 8.75   
            300+                2022-09-15   13.00 10.88 2.42 6.00 8.20 9.90   
sudak       30-50               2022-09-15    6.00  5.33 3.21 2.80 2.90 3.05   
            50-70               2022-09-15    8.00  5.99 3.16 4.00 4.00 4.15   
            70-150              2022-09-15   41.00  7.25 2.62 1.50 4.20 5.40   
            150-300             2022-09-15   22.00  8.94 3.81 2.00 4.30 5.35   
            300+                2022-09-15    3.00  9.33 0.58 9.00 9.00 9.00   

                                             50%   75%   90%   max  
place       house_size_category dt                                  
bahchisaray 30-50               2022-09-15  3.40  4.00  4.64  7.90  
            50-70               2022-09-15  4.05  5.70  6.94 10.50  
            70-150              2022-09-15  5.50  7.50 10.15 14.25  
            150-300             2022-09-15  7.70 11.25 12.50 14.99  
            300+                2022-09-15  8.95 12.22 13.25 13.50  
sevastopol  30-50               2022-09-15  4.99  6.00  7.50 11.60  
            50-70               2022-09-15  5.60  7.30  9.00 14.90  
            70-150              2022-09-15  7.90  9.50 12.00 14.91  
            150-300             2022-09-15 10.85 12.90 14.00 14.95  
            300+                2022-09-15 10.50 12.70 13.36 14.90  
sudak       30-50               2022-09-15  4.10  6.50  9.00 11.00  
            50-70               2022-09-15  4.95  6.03  8.53 13.50  
            70-150              2022-09-15  7.00  9.00 10.80 13.00  
            150-300             2022-09-15  9.60 12.07 12.98 14.80  
            300+                2022-09-15  9.00  9.50  9.80 10.00

In [16]:
# stat.loc[('bahchisaray','30-50'),['min','25%','50%']].plot(grid=True, title='цена на маленькие дома')

In [17]:
# stat.loc['30-50',['min','25%','50%']].plot(grid=True, title='цена на маленькие дома')

In [18]:
# stat.loc['50-70',['min','25%','50%']].plot(grid=True, title='цена на средние дома')

In [19]:
# stat.loc['70-150',['min','25%','50%']].plot(grid=True, title='цена на большие дома')

## гексагональная сетка

In [20]:
places = gdf['place'].unique()
places

array(['sevastopol', 'bahchisaray', 'sudak'], dtype=object)

In [21]:
from shapely.geometry import MultiPoint

frame_geometry = gpd.GeoDataFrame([{ 
            'place':p, 
            'geometry':MultiPoint( gdf[gdf['place']==p]['geometry'].tolist() ).convex_hull, 
        } for p in places ]
    ,crs='EPSG:4326')

In [22]:
frame_geometry.explore('place',legend=True,cmap='rainbow')

In [23]:
import h3
from shapely.geometry import Polygon

def cover_grid_hexagonal(gmtr,place,cell_size=8,crs='EPSG:4326'):
    polygonise = lambda hex_id: Polygon( h3.h3_to_geo_boundary( hex_id, geo_json=True)  )
    hexs = list(h3.polyfill( gmtr.__geo_interface__, cell_size, geo_json_conformant = True ))
    return gpd.GeoDataFrame({'place':place,'hex_id': hexs, 'geometry':list(map(polygonise, hexs)), },crs=crs,)

In [24]:
places = { # размеры сетки для разных мест
'sevastopol':6,
'sudak':8,
'bahchisaray':6,
}

grid = gpd.GeoDataFrame(
    pd.concat([
        cover_grid_hexagonal(frame_geometry[ frame_geometry['place']==p ]['geometry'].values[0],p,cell_size=cs)
        for p,cs in places.items()
        ], ignore_index=True)
    ,crs='EPSG:4326'
)

grid.explore('place',cmap='rainbow')

### медиана актуальной цены

In [25]:
place = ['bahchisaray',]

grid_ =  grid[grid['place'].isin(place)]

grid_values = (
   grid_.sjoin( gdf[ (gdf['dt']>ts) & (gdf['priceM']<15.) ], how='inner',)
    .groupby(['hex_id'])
    ['priceM']
    .median()
    .reset_index()
)

grid_.merge(grid_values,on='hex_id').explore('priceM',cmap='rainbow') #'area_name', legend=True)

In [26]:
place = ['sevastopol',]

grid_ =  grid[grid['place'].isin(place)]

grid_values = (
   grid_.sjoin( gdf[ (gdf['dt']>ts) & (gdf['priceM']<15.) ], how='inner',)
    .groupby(['hex_id'])
    ['priceM']
    .median()
    .reset_index()
)

grid_.merge(grid_values,on='hex_id').explore('priceM',cmap='rainbow') #'area_name', legend=True)

In [27]:
place = ['sudak',]

grid_ =  grid[grid['place'].isin(place)]

grid_values = (
   grid_.sjoin( gdf[ (gdf['dt']>ts) & (gdf['priceM']<15.) ], how='inner',)
    .groupby(['hex_id'])
    ['priceM']
    .median()
    .reset_index()
)

grid_.merge(grid_values,on='hex_id').explore('priceM',cmap='rainbow') #'area_name', legend=True)

---